# Indexing Selecting Data Pandas

In [1]:
import pandas as pd
import numpy as np

The axis labeling information in pandas objects serves many purposes:

* Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display.
* Enables automatic and explicit data alignment.
* Allows intuitive getting and setting of subsets of the data set.

In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects. The primary focus will be on Series and DataFrame as they have received more development attention in this area.

__Note__:  The Python and NumPy indexing operators [] and attribute operator . provide quick and easy access to pandas data structures across a wide range of use cases. This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays. However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits. For production code, we recommended that you take advantage of the optimized pandas data access methods exposed in this chapter.

__Warning__: Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called chained assignment and should be avoided. See Returning a View versus Copy.


__Warning__: Indexing on an integer-based Index with floats has been clarified in 0.18.0, for a summary of the changes, see here.


See the [MultiIndex / Advanced Indexing for MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced) and more advanced indexing documentation.

See the [cookbook](https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-selection) for some advanced strategies.

# Different Choices for Indexing

Object selection has had a number of user-requested additions in order to support more explicit location based indexing. Pandas now supports three types of multi-axis indexing.

* `.loc` is primarily label based, but may also be used with a boolean array. *.loc* will raise KeyError when the items are not found. Allowed inputs are:

    * A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is __not__ an integer position along the index.).

    * A list or array of labels ['a', 'b', 'c'].

    * A slice object with labels 'a':'f' (Note that contrary to usual python slices, both the start and the stop are included, when present in the index! See Slicing with labels.).

    * A boolean array

    * A callable function with one argument (the calling Series, DataFrame or Panel) and that returns valid output for indexing (one of the above).

    *New in version 0.18.1.*

See more at [Selection by Label.](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-label)


* `.iloc` is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a boolean array. *.iloc* will raise IndexError if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing. (this conforms with Python/NumPy slice semantics). Allowed inputs are:

    * An integer e.g. 5.

    * A list or array of integers [4, 3, 0].

    * A slice object with ints 1:7.

    * A boolean array.

    * A callable function with one argument (the calling Series, DataFrame or Panel) and that returns valid output for indexing (one of the above).

    *New in version 0.18.1.*

See more at [Selection by Position](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-integer), [Advanced Indexing and Advanced Hierarchical](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced).

* `.loc`, `.iloc`, and also `[]` indexing can accept a callable as indexer. See more at [Selection By Callable](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-callable).

Getting values from an object with multi-axes selection uses the following notation (using .loc as an example, but the following applies to .iloc as well). Any of the axes accessors may be the null slice :. Axes left out of the specification are assumed to be :, e.g. `p.loc['a']` is equivalent to `p.loc['a',:,:]`.

# Basics

As mentioned when introducing the data structures in the last section, the primary function of indexing with `[]` (a.k.a. `__getitem__` for those familiar with implementing class behavior in Python) is selecting out lower-dimensional slices. The following table shows return type values when indexing pandas objects with `[]`: [See more details here](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-view-versus-copy).

Here we construct a simple time series data set to use for illustrating the indexing functionality:

In [3]:
dates= pd.date_range("1/1/2000", periods=8)
df= pd.DataFrame(np.random.randn(8, 4),
                index=dates, columns=["A","B","C", "D"])

In [4]:
df

,A,B,C,D
2000-01-01,2.603216,-0.534047,0.477456,-0.999043
2000-01-02,2.192901,-0.821476,-0.430178,-1.314656
2000-01-03,-0.266416,0.754446,-0.678859,-2.560836
2000-01-04,0.252939,0.551459,0.095879,0.157549
2000-01-05,-0.235144,-2.012519,0.209264,-1.736123
2000-01-06,0.714743,-2.393329,1.216134,2.570287
2000-01-07,0.820027,-2.249255,-2.371049,-1.099577
2000-01-08,0.360294,0.218496,-0.022373,-0.421956


In [5]:
panel= pd.Panel({"one": df, "two":df-df.mean()})

/home/tenghe/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2878: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
panel

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 8 (major_axis) x 4 (minor_axis)
Items axis: one to two
Major_axis axis: 2000-01-01 00:00:00 to 2000-01-08 00:00:00
Minor_axis axis: A to D

__Note__: None of the indexing functionality is time series specific unless specifically stated

Thus as per the above, we have the most basic indexing using `[]`

In [7]:
s= df["A"]

In [8]:
panel["two"]

,A,B,C,D
2000-01-01,1.797896,0.276731,0.665422,-0.323498
2000-01-02,1.387581,-0.010698,-0.242212,-0.639112
2000-01-03,-1.071736,1.565224,-0.490893,-1.885291
2000-01-04,-0.552381,1.362237,0.283845,0.833093
2000-01-05,-1.040464,-1.201741,0.397230,-1.060579
2000-01-06,-0.090577,-1.582551,1.404100,3.245832
2000-01-07,0.014707,-1.438477,-2.183083,-0.424033
2000-01-08,-0.445026,1.029274,0.165593,0.253589


You can pass a list of columns to [] to select columns in that order. If a column is not contained in the DataFrame, an exception will be raised. Multiple columns can also be set in this manner:

In [9]:
df

,A,B,C,D
2000-01-01,2.603216,-0.534047,0.477456,-0.999043
2000-01-02,2.192901,-0.821476,-0.430178,-1.314656
2000-01-03,-0.266416,0.754446,-0.678859,-2.560836
2000-01-04,0.252939,0.551459,0.095879,0.157549
2000-01-05,-0.235144,-2.012519,0.209264,-1.736123
2000-01-06,0.714743,-2.393329,1.216134,2.570287
2000-01-07,0.820027,-2.249255,-2.371049,-1.099577
2000-01-08,0.360294,0.218496,-0.022373,-0.421956


In [10]:
df[[ "B", "A"]]=df[["A", "B"]]

In [11]:
df

,A,B,C,D
2000-01-01,-0.534047,2.603216,0.477456,-0.999043
2000-01-02,-0.821476,2.192901,-0.430178,-1.314656
2000-01-03,0.754446,-0.266416,-0.678859,-2.560836
2000-01-04,0.551459,0.252939,0.095879,0.157549
2000-01-05,-2.012519,-0.235144,0.209264,-1.736123
2000-01-06,-2.393329,0.714743,1.216134,2.570287
2000-01-07,-2.249255,0.820027,-2.371049,-1.099577
2000-01-08,0.218496,0.360294,-0.022373,-0.421956


You may find this useful for applying a transform (in-place) to a subset of the columns.

__Warning__: pandas aligns all AXES when setting Series and DataFrame from `.loc`, and `.iloc`.

This will __not__ modify df because the column alignment is before value assignment.

In [12]:
df[["A", "B"]]

,A,B
2000-01-01,-0.534047,2.603216
2000-01-02,-0.821476,2.192901
2000-01-03,0.754446,-0.266416
2000-01-04,0.551459,0.252939
2000-01-05,-2.012519,-0.235144
2000-01-06,-2.393329,0.714743
2000-01-07,-2.249255,0.820027
2000-01-08,0.218496,0.360294


In [13]:
df.loc[:, ["B","A"]] = df[["A","B"]]

In [14]:
df[["A", "B"]]

,A,B
2000-01-01,-0.534047,2.603216
2000-01-02,-0.821476,2.192901
2000-01-03,0.754446,-0.266416
2000-01-04,0.551459,0.252939
2000-01-05,-2.012519,-0.235144
2000-01-06,-2.393329,0.714743
2000-01-07,-2.249255,0.820027
2000-01-08,0.218496,0.360294


The correct way to swap column values is by using raw values

In [15]:
#df.loc[:, ["B", "A"]] = df[["A", "B"]].to_numpy()

# Attributes Access

You may access an index on a Series, column on a DataFrame, and an item on a Panel directly as an attribute:

In [16]:
sa= pd.Series( [1,2,3], index=list('abc'))

In [17]:
dfa=df.copy()

In [18]:
sa.b

2

In [19]:
df.A

2000-01-01   -0.534047
2000-01-02   -0.821476
2000-01-03    0.754446
2000-01-04    0.551459
2000-01-05   -2.012519
2000-01-06   -2.393329
2000-01-07   -2.249255
2000-01-08    0.218496
Freq: D, Name: A, dtype: float64

In [20]:
panel.one

,A,B,C,D
2000-01-01,2.603216,-0.534047,0.477456,-0.999043
2000-01-02,2.192901,-0.821476,-0.430178,-1.314656
2000-01-03,-0.266416,0.754446,-0.678859,-2.560836
2000-01-04,0.252939,0.551459,0.095879,0.157549
2000-01-05,-0.235144,-2.012519,0.209264,-1.736123
2000-01-06,0.714743,-2.393329,1.216134,2.570287
2000-01-07,0.820027,-2.249255,-2.371049,-1.099577
2000-01-08,0.360294,0.218496,-0.022373,-0.421956


In [21]:
panel.two

,A,B,C,D
2000-01-01,1.797896,0.276731,0.665422,-0.323498
2000-01-02,1.387581,-0.010698,-0.242212,-0.639112
2000-01-03,-1.071736,1.565224,-0.490893,-1.885291
2000-01-04,-0.552381,1.362237,0.283845,0.833093
2000-01-05,-1.040464,-1.201741,0.397230,-1.060579
2000-01-06,-0.090577,-1.582551,1.404100,3.245832
2000-01-07,0.014707,-1.438477,-2.183083,-0.424033
2000-01-08,-0.445026,1.029274,0.165593,0.253589


In [22]:
sa.a=5

In [23]:
sa

a    5
b    2
c    3
dtype: int64

In [24]:
dfa.A= list(range(len(dfa.index)))  # ok if A already exists

In [25]:
dfa

,A,B,C,D
2000-01-01,0,2.603216,0.477456,-0.999043
2000-01-02,1,2.192901,-0.430178,-1.314656
2000-01-03,2,-0.266416,-0.678859,-2.560836
2000-01-04,3,0.252939,0.095879,0.157549
2000-01-05,4,-0.235144,0.209264,-1.736123
2000-01-06,5,0.714743,1.216134,2.570287
2000-01-07,6,0.820027,-2.371049,-1.099577
2000-01-08,7,0.360294,-0.022373,-0.421956


In [26]:
dfa['A']= list(range(len(dfa.index)))  #use this form to create a new column

In [27]:
dfa

,A,B,C,D
2000-01-01,0,2.603216,0.477456,-0.999043
2000-01-02,1,2.192901,-0.430178,-1.314656
2000-01-03,2,-0.266416,-0.678859,-2.560836
2000-01-04,3,0.252939,0.095879,0.157549
2000-01-05,4,-0.235144,0.209264,-1.736123
2000-01-06,5,0.714743,1.216134,2.570287
2000-01-07,6,0.820027,-2.371049,-1.099577
2000-01-08,7,0.360294,-0.022373,-0.421956


__Warning__:

* You can use this access only if the index element is a valid Python identifier, e.g. `s.1` is not allowed. [See here for an explanation of valid identifiers](https://docs.python.org/3/reference/lexical_analysis.html#identifiers).
* The attribute will not be available if it conflicts with an existing method name, e.g. `s.min` is not allowed.
* Similarly, the attribute will not be available if it conflicts with any of the following list: `index`, `major_axis`, `minor_axis`, `items`.
* In any of these cases, standard indexing will still work, e.g. `s['1']`, `s['min']`, and `s['index']` will access the corresponding element or column.

If you are using the IPython environment, you may also use tab-completion to see these accessible attributes.

You can also assign a `dict` to a row of a `DataFrame`:

In [28]:
x= pd.DataFrame( {'x':[1,2,3], 'y': [3,4,5]})

In [29]:
x

,x,y
0,1,3
1,2,4
2,3,5


In [30]:
x.iloc[1]

x    2
y    4
Name: 1, dtype: int64

In [31]:
x.loc[1]={'x':9, 'y':99}

In [32]:
x

,x,y
0,1,3
1,9,99
2,3,5


You can use attribute access to modify an existing element of a Series or column of a DataFrame, but be careful; if you try to use attribute access to create a new column, it creates a new attribute rather than a new column. In 0.21.0 and later, this will raise a `UserWarning`:

In [33]:
dff= pd.DataFrame( {'one':[1.,2.,3.]})

In [34]:
dff.two= [4,5,6]

/home/tenghe/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [35]:
dff

,one
0,1.0
1,2.0
2,3.0


In [36]:
dff['two']= [4,5,6]

In [37]:
dff

,one,two
0,1.0,4
1,2.0,5
2,3.0,6


# Slicing ranges

The most robust and consistent way of slicing ranges along arbitrary axes is described in the [Selection by Position](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-integer) section detailing the `.iloc` method. For now, we explain the semantics of slicing using the `[]` operator.

With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels:

In [38]:
s

2000-01-01   -0.534047
2000-01-02   -0.821476
2000-01-03    0.754446
2000-01-04    0.551459
2000-01-05   -2.012519
2000-01-06   -2.393329
2000-01-07   -2.249255
2000-01-08    0.218496
Freq: D, Name: A, dtype: float64

In [39]:
s[:5]  #Get the first 5 rows

2000-01-01   -0.534047
2000-01-02   -0.821476
2000-01-03    0.754446
2000-01-04    0.551459
2000-01-05   -2.012519
Freq: D, Name: A, dtype: float64

In [40]:
s[::2] #Get every second row

2000-01-01   -0.534047
2000-01-03    0.754446
2000-01-05   -2.012519
2000-01-07   -2.249255
Freq: 2D, Name: A, dtype: float64

In [41]:
s[::-1]  # reverse rows

2000-01-08    0.218496
2000-01-07   -2.249255
2000-01-06   -2.393329
2000-01-05   -2.012519
2000-01-04    0.551459
2000-01-03    0.754446
2000-01-02   -0.821476
2000-01-01   -0.534047
Freq: -1D, Name: A, dtype: float64

Note that setting works as well

In [42]:
s2= s.copy()

In [43]:
s2[:5]= 0;s2

2000-01-01    0.000000
2000-01-02    0.000000
2000-01-03    0.000000
2000-01-04    0.000000
2000-01-05    0.000000
2000-01-06   -2.393329
2000-01-07   -2.249255
2000-01-08    0.218496
Freq: D, Name: A, dtype: float64

With DataFrame, slicing inside of `[]` __slices the rows__. This is provided largely as a convenience since it is such a common operation.

In [44]:
dfa[:3]

,A,B,C,D
2000-01-01,0,2.603216,0.477456,-0.999043
2000-01-02,1,2.192901,-0.430178,-1.314656
2000-01-03,2,-0.266416,-0.678859,-2.560836


In [45]:
dfa[::-1]

,A,B,C,D
2000-01-08,7,0.360294,-0.022373,-0.421956
2000-01-07,6,0.820027,-2.371049,-1.099577
2000-01-06,5,0.714743,1.216134,2.570287
2000-01-05,4,-0.235144,0.209264,-1.736123
2000-01-04,3,0.252939,0.095879,0.157549
2000-01-03,2,-0.266416,-0.678859,-2.560836
2000-01-02,1,2.192901,-0.430178,-1.314656
2000-01-01,0,2.603216,0.477456,-0.999043


# Selecting By Label

__Warning__: Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called `chained assignment` and should be avoided. See __Returning a View versus Copy__.

__Warning__:

`.loc` is strict when you present slicers that are not compatible (or convertible) with the index type. For example using integers in a `DatetimeIndex`. These will raise a `TypeError`.

In [46]:
df1= pd.DataFrame(np.random.randn(5,4),
                 columns=list("ABCD"),
                 index=pd.date_range("20130101", periods=5))

In [47]:
df1

,A,B,C,D
2013-01-01,0.680031,0.319548,0.730567,-0.496549
2013-01-02,0.420150,-0.294491,1.299552,-0.664397
2013-01-03,-1.258633,-0.029697,-0.209992,-1.124429
2013-01-04,-1.095744,0.122886,1.035413,-0.929717
2013-01-05,-0.303362,-0.329285,0.662032,-0.212254


In [48]:
#df1.loc[2:3]

`TypeError: cannot do slice indexing on <class 'pandas.core.indexes.datetimes.DatetimeIndex'> with these indexers [2] of <class 'int'> `

String likes in slicing can be convertible to the type of the index and lead to natural slicin

In [49]:
df1.loc['20130102': '20130104']

,A,B,C,D
2013-01-02,0.420150,-0.294491,1.299552,-0.664397
2013-01-03,-1.258633,-0.029697,-0.209992,-1.124429
2013-01-04,-1.095744,0.122886,1.035413,-0.929717


__Warning__: Starting in 0.21.0, pandas will show a `FutureWarning` if indexing with a list with missing labels. In the future this will raise a `KeyError`. See [list-like Using loc with missing keys in a list is Deprecated](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-deprecate-loc-reindex-listlike).

pandas provides a suite of methods in order to have __purely label based indexing__. This is a strict inclusion based protocol. Every label asked for must be in the index, or a `KeyError` will be raised. When slicing, both the start bound __AND__ the stop bound are included, if present in the index. Integers are valid labels, but they refer to the label and not the position.

The `.loc` attribute is the primary access method. The following are valid inputs:

* A single label, e.g. `5` or `'a'` (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).
* A list or array of labels `['a', 'b', 'c']`.
* A slice object with labels 'a':'f' (Note that contrary to usual python slices, __both__ the start and the stop are included, when present in the index! See Slicing with labels.).
* A boolean array.
* A `callable`, see [Selection By Callable](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-callable).

In [50]:
s1= pd.Series(np.random.randn(6), index=list("abcdef"))

In [51]:
s1

a    0.275313
b    0.960070
c   -0.616441
d   -0.650510
e    0.328674
f   -0.904994
dtype: float64

In [52]:
s1.loc['c']

-0.6164410597480486

In [53]:
s1.loc['b']

0.9600700670851942

In [54]:
s1[['a', 'c', 'd']]

a    0.275313
c   -0.616441
d   -0.650510
dtype: float64

Note that setting works as well:

In [55]:
s1.loc['c':]=0

In [56]:
s1

a    0.275313
b    0.960070
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

With a DataFrame

In [57]:
df1= pd.DataFrame(np.random.randn(6,4),
                 index=list('abcdef'),
                 columns=list("ABCD"))

In [58]:
df1

,A,B,C,D
a,0.133282,1.731406,-0.380633,-0.375357
b,0.459820,-0.658240,-2.099836,0.052732
c,-0.921016,-1.528586,-1.506790,-0.058805
d,-1.288850,0.635015,0.832605,-0.214760
e,-0.481565,-1.017262,2.191837,-0.630470
f,-0.541273,1.847189,-2.378323,0.026559


In [59]:
df1.loc[['a', 'b', 'd'], :]

,A,B,C,D
a,0.133282,1.731406,-0.380633,-0.375357
b,0.459820,-0.658240,-2.099836,0.052732
d,-1.288850,0.635015,0.832605,-0.214760


In [60]:
df1.loc[['a', 'b', 'd'], ['A', 'D']]

,A,D
a,0.133282,-0.375357
b,0.459820,0.052732
d,-1.288850,-0.214760


Accessing vial label slices

In [61]:
df1.loc['d':, 'A':'C']

,A,B,C
d,-1.288850,0.635015,0.832605
e,-0.481565,-1.017262,2.191837
f,-0.541273,1.847189,-2.378323


For getting a cross section using a label (equivalent to `df.xs('a')`):

In [62]:
df1.loc['a']

A    0.133282
B    1.731406
C   -0.380633
D   -0.375357
Name: a, dtype: float64

For getting  values with a boolen array

In [63]:
df1.loc['a']>0

A     True
B     True
C    False
D    False
Name: a, dtype: bool

In [64]:
df1

,A,B,C,D
a,0.133282,1.731406,-0.380633,-0.375357
b,0.459820,-0.658240,-2.099836,0.052732
c,-0.921016,-1.528586,-1.506790,-0.058805
d,-1.288850,0.635015,0.832605,-0.214760
e,-0.481565,-1.017262,2.191837,-0.630470
f,-0.541273,1.847189,-2.378323,0.026559


In [65]:
df1.loc[:, df1.loc['a']>0] # Get the column for which row a is > 0

,A,B
a,0.133282,1.731406
b,0.459820,-0.658240
c,-0.921016,-1.528586
d,-1.288850,0.635015
e,-0.481565,-1.017262
f,-0.541273,1.847189


For getting a value explicitly (equivalent to deprcated `df.get_value('a', 'A')`

In [66]:
#this is also equivalent to `df1.at['a','A']`
df1.loc['a', 'A']

0.1332823859917978

# Slicing with labels

When using .loc with slices, if both the start and the stop labels are present in the index, then elements located between the two (including them) are returned:

In [67]:
s= pd.Series(list('abcde'), index=[0,3,2,5,4])

In [68]:
s

0    a
3    b
2    c
5    d
4    e
dtype: object

In [69]:
s.loc[3:5]

3    b
2    c
5    d
dtype: object

If at least one of the two is absent, but the index is sorted, and can be compared against start and stop labels, then slicing will still work as expected, by selecting labels which *rank* between the two:

In [70]:
s.sort_index()

0    a
2    c
3    b
4    e
5    d
dtype: object

In [71]:
s.sort_index().loc[1:6]

2    c
3    b
4    e
5    d
dtype: object

However, if at least one of the two is absent and the index is not sorted, an error will be raised (since doing otherwise would be computationally expensive, as well as potentially ambiguous for mixed type indexes). For instance, in the above example, `s.loc[1:6]` would raise `KeyError`.

# Selection By Position

__Warning__: Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called `chained assignment` and should be avoided. See __Returning a View versus Copy__.

Pandas provides a suite of methods in order to get __purely integer based indexing__. The semantics follow closely Python and NumPy slicing. These are `0-based` indexing. When slicing, the start bounds is included, while the upper bound is excluded. Trying to use a non-integer, even a valid label will raise an `IndexError`.

The `.iloc` attribute is the primary access method. The following are valid inputs:

* An integer e.g. `5`.
* A list or array of integers `[4, 3, 0]`.
* A slice object with ints `1:7`.
* A boolean array.
* A `callable`, see Selection By Callable.

In [72]:
s1= pd.Series(np.random.randn(5), index=list(range(0,10, 2)))

In [73]:
s1

0    0.985940
2    0.450258
4   -0.691279
6    1.583001
8    0.924290
dtype: float64

In [74]:
s1.iloc[:3]   #Get the first 3 elements

0    0.985940
2    0.450258
4   -0.691279
dtype: float64

In [75]:
s1.iloc[3]  #Get value in position 4

1.5830011499502443

Note that setting works as well

In [76]:
s1.iloc[:3]=0

In [77]:
s1

0    0.000000
2    0.000000
4    0.000000
6    1.583001
8    0.924290
dtype: float64

With a DataFrame:

In [78]:
df1= pd.DataFrame(np.random.randn(6,4), 
                 index=list(range(0, 12,2)),
                 columns=list(range(0,8,2)))

In [79]:
df1 

,0,2,4,6
0,-1.420831,0.129095,0.839202,0.394467
2,1.531849,-1.446379,-1.179841,1.986686
4,-1.022505,-1.119284,-1.529131,0.199323
6,-0.039760,1.762673,-0.730686,-0.580164
8,-2.021309,-1.471721,-0.333055,0.600822
10,-1.354469,0.179407,2.416819,0.821257


Select via integer slicing:

In [80]:
df1.iloc[:3] #Get first 3 rows for all columns

,0,2,4,6
0,-1.420831,0.129095,0.839202,0.394467
2,1.531849,-1.446379,-1.179841,1.986686
4,-1.022505,-1.119284,-1.529131,0.199323


In [81]:
df1.iloc[1:5, 2:4] #Get rows 1to 5 of cols 2 to 4

,4,6
2,-1.179841,1.986686
4,-1.529131,0.199323
6,-0.730686,-0.580164
8,-0.333055,0.600822


In [82]:
df1.iloc[ :, 2]  #Get cols 2

0     0.839202
2    -1.179841
4    -1.529131
6    -0.730686
8    -0.333055
10    2.416819
Name: 4, dtype: float64

In [83]:
df1.iloc[ : , 1:3]  #Get cols 1 and 2

,2,4
0,0.129095,0.839202
2,-1.446379,-1.179841
4,-1.119284,-1.529131
6,1.762673,-0.730686
8,-1.471721,-0.333055
10,0.179407,2.416819


Select via integer list;

In [84]:
df1.iloc[[1,3,5], [1,3]]

,2,6
2,-1.446379,1.986686
6,1.762673,-0.580164
10,0.179407,0.821257


In [85]:

df1.iloc[1:3, :]

,0,2,4,6
2,1.531849,-1.446379,-1.179841,1.986686
4,-1.022505,-1.119284,-1.529131,0.199323


In [86]:
df1.iloc[:, 1:3]

,2,4
0,0.129095,0.839202
2,-1.446379,-1.179841
4,-1.119284,-1.529131
6,1.762673,-0.730686
8,-1.471721,-0.333055
10,0.179407,2.416819


In [87]:
# this is also equivalent to ``df1.iat[1,1]``
df1.iloc[1,1]

-1.4463786135345413

For getting a cross section using an integer position (equiv to `df.xs(1)`):

In [88]:
df1.iloc[1]

0    1.531849
2   -1.446379
4   -1.179841
6    1.986686
Name: 2, dtype: float64

Out of range slice indexes are handled gracefully just as in Python/Numpy.

In [89]:
# these are allowed in Python/Numpy
x= list('abcdef'); x

['a', 'b', 'c', 'd', 'e', 'f']

In [90]:
x[4:10]

['e', 'f']

In [91]:
x[8:10]

[]

In [92]:
s= pd.Series(x)

In [93]:
s

0    a
1    b
2    c
3    d
4    e
5    f
dtype: object

In [94]:
s.iloc[4:10]

4    e
5    f
dtype: object

In [95]:
s.iloc[8:10]

Series([], dtype: object)

Note that using slices that go out of bounds can result in an empty axis (e.g. an empty DataFrame being returned).

In [96]:
df1= pd.DataFrame(np.random.randn(5,2),
                  columns=list('AB'))

In [97]:
df1

,A,B
0,-0.709004,-0.088186
1,-1.276262,-0.831705
2,0.672240,-1.065555
3,0.924963,1.599709
4,-0.591647,2.267590


In [98]:
df1.iloc[:, 2:3]

""
0
1
2
3
4


In [99]:
df1.iloc[:, 1:3]

,B
0,-0.088186
1,-0.831705
2,-1.065555
3,1.599709
4,2.267590


In [100]:
df1.iloc[4:6]

,A,B
4,-0.591647,2.26759


A single indexer that is out of bounds will raise an `IndexError`. A list of indexers where any element is out of bounds will raise an `IndexError`.

` >>> df1.iloc[[4,5,6]]`

``IndexError: positional indexers are out-of-bounds``


` >>> df1.iloc[:, 4]`

`IndexError: single positional indexer is out-of-bounds`

# Selection By callable

*New in version 0.18.1.*

`.loc`, `.iloc`, and also `[]` indexing can accept a `callable` as indexer. The `callable` __must be a function with one argument__ (the calling Series, DataFrame or Panel) __and that returns valid output for indexing__.

In [101]:
df1= pd.DataFrame(np.random.randn(6,4),
                 index=list('abcdef'),
                 columns=list("ABCD"))

In [102]:
df1

,A,B,C,D
a,0.968773,-0.944432,-0.249347,1.483217
b,0.170370,0.704072,1.765541,1.732772
c,-0.001135,-1.057906,-0.831277,1.363059
d,-0.910482,0.145376,-0.751715,-1.430421
e,0.437796,-0.955571,-0.449271,-0.673238
f,-0.352880,-0.309813,-0.351201,1.695626


In [103]:
df1.loc[lambda df: df.A > 0, :]

,A,B,C,D
a,0.968773,-0.944432,-0.249347,1.483217
b,0.170370,0.704072,1.765541,1.732772
e,0.437796,-0.955571,-0.449271,-0.673238


In [104]:
df1.loc[:, lambda df: ['A', 'B'] ]

,A,B
a,0.968773,-0.944432
b,0.170370,0.704072
c,-0.001135,-1.057906
d,-0.910482,0.145376
e,0.437796,-0.955571
f,-0.352880,-0.309813


In [105]:
df1.loc[:, ['A', 'B']]

,A,B
a,0.968773,-0.944432
b,0.170370,0.704072
c,-0.001135,-1.057906
d,-0.910482,0.145376
e,0.437796,-0.955571
f,-0.352880,-0.309813


In [106]:
df1.iloc[:, lambda df: [0,1]]

,A,B
a,0.968773,-0.944432
b,0.170370,0.704072
c,-0.001135,-1.057906
d,-0.910482,0.145376
e,0.437796,-0.955571
f,-0.352880,-0.309813


In [280]:
df1[lambda df: df.columns[0]] #Get values of column 0 of DataFrame

a    0.968773
b    0.170370
c   -0.001135
d   -0.910482
e    0.437796
f   -0.352880
Name: A, dtype: float64

You can use `callable` indexing in *Series*

In [281]:
df1.A.loc[lambda s: s > 0]

a    0.968773
b    0.170370
e    0.437796
Name: A, dtype: float64

Using these methods / indexers, you can chain data selection operations without using temporary variable.

`bb = pd.read_csv('data/baseball.csv', index_col='id')`
` (bb.groupby(['year', 'team']).sum()
         .loc[lambda df: df.r > 100])`

# IX Indexer is Deprecated

__Warning__: Starting in 0.20.0, the `.ix` indexer is deprecated, in favor of the more strict `.iloc` and `.loc` indexers.

`.ix` offers a lot of magic on the inference of what the user wants to do. To wit, `.ix` can decide to index positionally OR via labels depending on the data type of the index. This has caused quite a bit of user confusion over the years.

The recommended methods of indexing are:

* `.loc` if you want to label index.
* `.iloc` if you want to positionally index.

In [109]:
dfd= pd.DataFrame( {'A':[1,2,3],
                    'B': [4,5,6]},
                    index=list('abc'))

In [110]:
dfd

,A,B
a,1,4
b,2,5
c,3,6


Previous behavior, where you wish to get the 0th and the 2nd elements from the index in the ‘A’ column.

`dfd.ix[[0,2], 'A']`

`a    1
c    3
Name: A, dtype: int64`

Using `.loc`. Here we will select the appropriate indexes from the index, then use label indexing.

In [111]:
dfd.loc[dfd.index[[0,2]], 'A'] # Using indexes (rows) and labels(columns) for subsetting

a    1
c    3
Name: A, dtype: int64

This can also be expressed using .iloc, by explicitly getting locations on the indexers, and using positional indexing to select things.

In [112]:
dfd.iloc[[0,2], dfd.columns.get_loc('A')]

a    1
c    3
Name: A, dtype: int64

In [113]:
dfd.iloc[[0,2], ]

,A,B
a,1,4
c,3,6


For getting *multiple* indexers, using `.get_indexer`:

In [114]:
dfd.iloc[[0,2], dfd.columns.get_indexer(['A','B'])]

,A,B
a,1,4
c,3,6


# Indexing with list with missing labels is Deprecated

__Warning__:  Starting in 0.21.0, using `.loc` or `[]` with a list with one or more missing labels, is deprecated, in favor of `.reindex`.

In prior versions, using `.loc[list-of-labels]` would work as long as *at least 1* of the keys was found (otherwise it would raise a `KeyError`). This behavior is deprecated and will show a warning message pointing to this section. The recommended alternative is to use `.reindex()`.

For example.

In [115]:
s= pd.Series([1,2,3])

In [116]:
s

0    1
1    2
2    3
dtype: int64

Selection with all keys found is unchanged.

In [117]:
s.loc[[1,2]]

1    2
2    3
dtype: int64

Previous Behavior

`s.loc[[1,2,3]]`

Passing list-likes to `.loc` with any non-matching elements will raise
`KeyError` in the future, you can use `.reindex()` as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike

`
1    2.0
2    3.0
3    NaN
dtype: float64`

# Reindexing

The idiomatic way to achieve selecting potentially not-found elements is via `.reindex()`. See also the section on [reindexing](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#basics-reindexing).

In [118]:
s

0    1
1    2
2    3
dtype: int64

In [119]:
s.reindex([1,2,3])

1    2.0
2    3.0
3    NaN
dtype: float64

Alternatively, if you want to select only valid keys, the following is idiomatic and efficient; it is guaranteed to preserve the dtype of the selection.

In [120]:
labels= [1,2,3]

In [121]:
s.loc[s.index.intersection(labels)]

1    2
2    3
dtype: int64

Having a duplicated index will raise `Error` for a `.reindex()`:

In [122]:
s=pd.Series(np.arange(4), index=['a','a','b','c'])

In [123]:
labels=['c', 'd']

`>>> s.reindex(labels)`

`ValueError: cannot reindex from a duplicate axis`

Generally, you can intersect the desired labels with the current axis, and then reindex.

In [124]:
s.loc[s.index.intersection(labels)].reindex(labels)

c    3.0
d    NaN
dtype: float64

However, this would still raise `Error` if your resulting index is duplicated.

In [125]:
labels = ['a', 'd']

`>>> s.loc[s.index.intersection(labels)].reindex(labels)`

`ValueError: cannot reindex from a duplicate axis` 

# Selecting Random Samples

A random selection of rows or columns from a Series, DataFrame, or Panel with the [sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html#pandas.DataFrame.sample) method. The method will sample rows by default, and accepts a specific number of rows/columns to return, or a fraction of rows.

In [126]:
s= pd.Series( range(6))

In [127]:
s

0    0
1    1
2    2
3    3
4    4
5    5
dtype: int64

In [128]:
# When no arguments are passed, returns 1 row
s.sample()

2    2
dtype: int64

In [129]:
# One may specify either a number of rows
s.sample(n=3)

0    0
1    1
5    5
dtype: int64

In [130]:
# Or a fraction of the rows
s.sample(frac=0.5)

3    3
4    4
5    5
dtype: int64

By default, `sample` will return each row at most once, but one can also sample with replacement using the `replace` option:

In [131]:
s= pd.Series(range(6))

In [132]:
# Without replacement (default)
s.sample(n=6, replace=False)

1    1
0    0
2    2
4    4
5    5
3    3
dtype: int64

In [133]:
# With replacement:
s.sample(n=6, replace=True)

5    5
1    1
4    4
2    2
1    1
0    0
dtype: int64

By default, each row has an equal probability of being selected, but if you want rows to have different probabilities, you can pass the sample function sampling weights as weights. These weights can be a list, a NumPy array, or a Series, but they must be of the same length as the object you are sampling. Missing values will be treated as a weight of zero, and inf values are not allowed. If weights do not sum to 1, they will be re-normalized by dividing all weights by the sum of the weights. For example:

In [134]:
s= pd.Series([0,1,2,3,4,5])

In [135]:
example_weights= [0,0,0.2,0.2,0.2,0.4]

In [136]:
s.sample(n=3, weights=example_weights)

4    4
3    3
5    5
dtype: int64

In [137]:
# Weights will be re-normalized automatically
example_weights2= [0.5, 0, 0, 0, 0, 0]

In [138]:
s.sample(n=1,  weights=example_weights2)

0    0
dtype: int64

When applied to a DataFrame, you can use a column of the DataFrame as sampling weights (provided you are sampling rows and not columns) by simply passing the name of the column as a string.

In [139]:
df2= pd.DataFrame( {'col1': [9,8,7,6],
                   'weight_column': [.5,.4,.1,0]})

In [140]:
df2.sample(n=3, weights='weight_column')

,col1,weight_column
0,9,0.5
1,8,0.4
2,7,0.1


`sample` also allows users to sample columns instead of rows using the `axis`argument

In [141]:
df3= pd.DataFrame( {'col1': [1,2,3], 'col2': [2,3,4]})

In [142]:
df3

,col1,col2
0,1,2
1,2,3
2,3,4


In [143]:
df3.sample(n=1, axis=1)

,col1
0,1
1,2
2,3


Finally, one can also set a __seed__ for `sample’s` random number generator using the `random_state argument`, which will accept either an integer (as a seed) or a NumPy RandomState object.

In [144]:
df4= pd.DataFrame({'col1': [1,2,3], 'col2': [2,3,4]})

In [145]:
df4

,col1,col2
0,1,2
1,2,3
2,3,4


In [146]:
# With a given seed, the sample will always draw the same rows.
df4.sample(n=2, random_state=2)

,col1,col2
2,3,4
1,2,3


In [147]:
df4.sample(n=2, random_state=2)

,col1,col2
2,3,4
1,2,3


# Setting With Enlargement

The `.loc/[]` operations can perform enlargement when setting a non-existent key for that axis.

In the *Series* case this is effectively an appending operation.

In [148]:
se= pd.Series([1,2,3])

In [149]:
se

0    1
1    2
2    3
dtype: int64

In [150]:
se[5]= 5

In [151]:
se

0    1
1    2
2    3
5    5
dtype: int64

A *DataFrame* can be enlarged on either axis via `.loc`

In [152]:
dfi= pd.DataFrame(np.arange(6).reshape(3,2),
                 columns=['A', 'B'])

In [153]:
dfi

,A,B
0,0,1
1,2,3
2,4,5


In [154]:
dfi.loc[:, 'C']= dfi.loc[:, 'A']

In [155]:
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4


This is like an append on the *DataFrame*

In [156]:
dfi.loc[3]=5  # Add row with index 3, an value=5 for all columns

In [157]:
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4
3,5,5,5


# Fast scalar value getting and setting

Since indexing with `[]` must handle a lot of cases (single-label access, slicing, boolean indexing, etc.), it has a bit of overhead in order to figure out what you’re asking for. If you only want to access a scalar value, the fastest way is to use the `at` and `iat` methods, which are implemented on all of the data structures.

Similarly to `loc`, `at` provides label based scalar lookups, while, `iat` provides integer based lookups analogously to `iloc`

In [166]:
s

0    0
1    1
2    2
3    3
4    4
5    5
dtype: int64

In [167]:
s.iat[5]

5

In [168]:
df

,A,B,C,D
2000-01-01,-0.534047,2.603216,0.477456,-0.999043
2000-01-02,-0.821476,2.192901,-0.430178,-1.314656
2000-01-03,0.754446,-0.266416,-0.678859,-2.560836
2000-01-04,0.551459,0.252939,0.095879,0.157549
2000-01-05,-2.012519,-0.235144,0.209264,-1.736123
2000-01-06,-2.393329,0.714743,1.216134,2.570287
2000-01-07,-2.249255,0.820027,-2.371049,-1.099577
2000-01-08,0.218496,0.360294,-0.022373,-0.421956


In [170]:
df.at[dates[5], 'A']

-2.393329035108446

In [171]:
df.iat[3,0]

0.5514589505766991

You can also set using these same indexes

In [172]:
df.at[dates[5], 'E'] = 7

In [174]:
df.iat[3,0] = 7

In [175]:
df

,A,B,C,D,E
2000-01-01,-0.534047,2.603216,0.477456,-0.999043,NaN
2000-01-02,-0.821476,2.192901,-0.430178,-1.314656,NaN
2000-01-03,0.754446,-0.266416,-0.678859,-2.560836,NaN
2000-01-04,7.000000,0.252939,0.095879,0.157549,NaN
2000-01-05,-2.012519,-0.235144,0.209264,-1.736123,NaN
2000-01-06,-2.393329,0.714743,1.216134,2.570287,7.0
2000-01-07,-2.249255,0.820027,-2.371049,-1.099577,NaN
2000-01-08,0.218496,0.360294,-0.022373,-0.421956,NaN


`at` may enlarge the object in-place as above if the indexer is missing.

In [176]:
df.at[dates[-1] + 1, 0] =7

In [177]:
df

,A,B,C,D,E,0
2000-01-01,-0.534047,2.603216,0.477456,-0.999043,NaN,NaN
2000-01-02,-0.821476,2.192901,-0.430178,-1.314656,NaN,NaN
2000-01-03,0.754446,-0.266416,-0.678859,-2.560836,NaN,NaN
2000-01-04,7.000000,0.252939,0.095879,0.157549,NaN,NaN
2000-01-05,-2.012519,-0.235144,0.209264,-1.736123,NaN,NaN
2000-01-06,-2.393329,0.714743,1.216134,2.570287,7.0,NaN
2000-01-07,-2.249255,0.820027,-2.371049,-1.099577,NaN,NaN
2000-01-08,0.218496,0.360294,-0.022373,-0.421956,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN,NaN,7.0


# Boolen indexing

Another common operation is the use of boolean vectors to filter the data. The operators are: `| for or`, `& for and`, and `~ for not`. These must be grouped by using parentheses, since by default Python will evaluate an expression such as `df.A > 2 & df.B < 3 as df.A > (2 & df.B) < 3`, while the desired evaluation order is `(df.A > 2) & (df.B < 3)`.

Using a boolean vector to index a Series works exactly as in a NumPy ndarray:

In [179]:
s= pd.Series(range(-3, 4)); s

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [180]:
s[s > 0]

4    1
5    2
6    3
dtype: int64

In [181]:
s[(s < -1) | (s>.5)]

0   -3
1   -2
4    1
5    2
6    3
dtype: int64

In [182]:
s[ ~(s>0)]

0   -3
1   -2
2   -1
3    0
dtype: int64

In [183]:
s[~(s<0)]

3    0
4    1
5    2
6    3
dtype: int64

You may select rows from a DataFrame using a boolean vector the same length as the DataFrame’s index (for example, something derived from one of the columns of the DataFrame):

In [184]:
df[df['A'] > 0]

,A,B,C,D,E,0
2000-01-03,0.754446,-0.266416,-0.678859,-2.560836,NaN,NaN
2000-01-04,7.000000,0.252939,0.095879,0.157549,NaN,NaN
2000-01-08,0.218496,0.360294,-0.022373,-0.421956,NaN,NaN


List comprehensions and `map` method of Series can also be used to produce more complex criteria:

In [186]:
df2= pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                  'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                  'c': np.random.randn(7)})

In [193]:
df2.head()

,a,b,c
0,one,x,1.080003
1,one,y,-1.396077
2,two,y,-0.522307
3,three,x,1.349570
4,two,y,2.245795


In [191]:
# Only two or three
criterion= df2['a'].map(lambda x: x.startswith('t'))
df2[criterion]

,a,b,c
2,two,y,-0.522307
3,three,x,1.349570
4,two,y,2.245795


In [197]:
filters= df2.a.map(lambda x: x.startswith('t'))
df2[filters]

,a,b,c
2,two,y,-0.522307
3,three,x,1.349570
4,two,y,2.245795


In [190]:
# equivalent but slower
df2[ [x.startswith('t') for x in df2['a']] ]

,a,b,c
2,two,y,-0.522307
3,three,x,1.349570
4,two,y,2.245795


In [195]:
df2[ [ x.startswith('t') for x in df2.a]]

,a,b,c
2,two,y,-0.522307
3,three,x,1.349570
4,two,y,2.245795


In [200]:
# Multiple criteria
df2[criterion & (df2['b']=='y')]

,a,b,c
2,two,y,-0.522307
4,two,y,2.245795


With the choice methods __Selection by Label, Selection by Position__, and __Advanced Indexing__ you may select along more than one axis using boolean vectors combined with other indexing expressions.

In [201]:
df2.loc[criterion & (df2['b']=='y'), 'b': 'c']

,b,c
2,y,-0.522307
4,y,2.245795


__Note__: For list comprehension and `map` method of *Series*, the filters are applied on slected columns and looped over each row of a DataFrame

# Indexing with isin

Consider the [isin()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.isin.html#pandas.Series.isin) method of Series, which returns a boolean vector that is true wherever the Series elements exist in the passed list. This allows you to select rows where one or more columns have values you want:

In [203]:
s = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')

In [204]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [205]:
s.isin([2,4,6])

4    False
3    False
2     True
1    False
0     True
dtype: bool

In [208]:
s[s.isin([2, 4, 5])]

2    2
0    4
dtype: int64

In [209]:
s[ ~ (s.isin([2,4,5]))]

4    0
3    1
1    3
dtype: int64

The same method is available for `Index` objects and is useful for the cases when you don’t know which of the sought labels are in fact present:

In [210]:
s[s.index.isin([2, 4, 6])]

4    0
2    2
dtype: int64

In [211]:
# compare it to the following
s.reindex([2,4,6])

2    2.0
4    0.0
6    NaN
dtype: float64

In addition to that, *MultiIndex*  allows selecting a separate level to use in the membership check:

In [212]:
s_mi= pd.Series(np.arange(6),
               index= pd.MultiIndex.from_product([ [0,1], ['a','b','c']]))

In [213]:
s_mi

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int64

In [214]:
s_mi.iloc[s_mi.index.isin( [(1,'a'), (2,'b'), (0,'c') ])]

0  c    2
1  a    3
dtype: int64

In [215]:
s_mi.iloc[s_mi.index.isin( ['a', 'c', 'e'], level= 1)]

0  a    0
   c    2
1  a    3
   c    5
dtype: int64

In [219]:
s_mi.iloc[s_mi.index.isin( ['a', 'c'], level=0)]

Series([], dtype: int64)

DataFrame also has an __isin()__ method. When calling isin, pass a set of values as either an array or dict. If values is an array, isin returns a DataFrame of booleans that is the same shape as the original DataFrame, with True wherever the element is in the sequence of values.

In [220]:
df= pd.DataFrame( {'vals': [1,2,3,4], 'ids': ['a', 'b', 'f', 'n'],
                  'ids2': ['a','n','c','n']})

In [221]:
df

,ids,ids2,vals
0,a,a,1
1,b,n,2
2,f,c,3
3,n,n,4


In [222]:
values= ['a', 'b', 1,3]

In [225]:
values

['a', 'b', 1, 3]

In [226]:
df.isin(values)

,ids,ids2,vals
0,True,True,True
1,True,False,False
2,False,False,True
3,False,False,False


Oftentimes you’ll want to match certain values with certain columns. Just make values a dict where the key is the column, and the value is a list of items you want to check for.

In [228]:
values= {'ids':['a','b'], 'vals': [1,3]}

In [229]:
df.isin(values)

,ids,ids2,vals
0,True,False,True
1,True,False,False
2,False,False,True
3,False,False,False


Combine DataFrame’s isin with the `any()` and `all()` methods to quickly select subsets of your data that meet a given criteria. To select a row where each column meets its own criterion:


In [230]:
values = {'ids': ['a', 'b'], 'ids2':['a', 'c'], 'vals':[1,3]}

In [241]:
row_mask= df.isin(values).all(1) # Apply filters to all rows 

In [242]:
row_mask

0     True
1    False
2    False
3    False
dtype: bool

In [243]:
df[row_mask]

,ids,ids2,vals
0,a,a,1


# The [where()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.where.html#pandas.DataFrame.where) Method and Masking

`where()`: Replace values where the condition is True.

Selecting values from a Series with a boolean vector generally returns a subset of the data. To guarantee that selection output has the same shape as the original data, you can use the `where` method in *Series* and *DataFrame*.

To return only the selected rows:

In [293]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [294]:
s[s > 0]

3    1
2    2
1    3
0    4
dtype: int64

To return a Series of the same shape as the original:

In [245]:
s.where(s > 0)

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

Selecting values from a DataFrame with a boolean criterion now also preserves input data shape. `where` is used 
under the hood as the implementation. The code below is equivalent to `df.where(df < 0).`

In [295]:
dates= pd.date_range("1/1/2000", periods=8)
df= pd.DataFrame(np.random.randn(8, 4),
                index=dates, columns=["A","B","C", "D"])

In [296]:
df[df < 0]

,A,B,C,D
2000-01-01,NaN,NaN,NaN,-0.404868
2000-01-02,-0.157870,NaN,-1.362580,-0.357375
2000-01-03,-0.016534,NaN,-0.990686,-1.797528
2000-01-04,-0.079916,-2.458044,NaN,NaN
2000-01-05,NaN,-2.486026,-0.805138,-0.743355
2000-01-06,NaN,NaN,NaN,-0.318010
2000-01-07,NaN,NaN,-0.348340,NaN
2000-01-08,-0.194843,-1.583846,-0.458041,-0.690173


In [297]:
df.where(df < 0)

,A,B,C,D
2000-01-01,NaN,NaN,NaN,-0.404868
2000-01-02,-0.157870,NaN,-1.362580,-0.357375
2000-01-03,-0.016534,NaN,-0.990686,-1.797528
2000-01-04,-0.079916,-2.458044,NaN,NaN
2000-01-05,NaN,-2.486026,-0.805138,-0.743355
2000-01-06,NaN,NaN,NaN,-0.318010
2000-01-07,NaN,NaN,-0.348340,NaN
2000-01-08,-0.194843,-1.583846,-0.458041,-0.690173


In addition, `where` takes an optional other argument for replacement of values where the condition is False, in the returned copy.

In [298]:
df.where(df < 0, -df)

,A,B,C,D
2000-01-01,-0.257600,-1.599861,-0.174971,-0.404868
2000-01-02,-0.157870,-0.072178,-1.362580,-0.357375
2000-01-03,-0.016534,-0.166315,-0.990686,-1.797528
2000-01-04,-0.079916,-2.458044,-1.164954,-0.631615
2000-01-05,-0.082609,-2.486026,-0.805138,-0.743355
2000-01-06,-2.567500,-1.026044,-0.653128,-0.318010
2000-01-07,-0.929976,-0.391964,-0.348340,-0.188350
2000-01-08,-0.194843,-1.583846,-0.458041,-0.690173


You may wish to set values based on some boolean criteria. This can be done intuitively like so:

In [299]:
s2= s.copy()

In [300]:
s2[s2 < 0] = 0

In [301]:
s2

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [302]:
dfn= df.copy()

In [303]:
dfn[dfn <0] =0

In [304]:
dfn

,A,B,C,D
2000-01-01,0.257600,1.599861,0.174971,0.000000
2000-01-02,0.000000,0.072178,0.000000,0.000000
2000-01-03,0.000000,0.166315,0.000000,0.000000
2000-01-04,0.000000,0.000000,1.164954,0.631615
2000-01-05,0.082609,0.000000,0.000000,0.000000
2000-01-06,2.567500,1.026044,0.653128,0.000000
2000-01-07,0.929976,0.391964,0.000000,0.188350
2000-01-08,0.000000,0.000000,0.000000,0.000000


By default, `where` returns a modified copy of the data. There is an optional parameter `inplace` so that the original data can be modified without creating a copy:

In [305]:
df_orig = df.copy()

In [306]:
df_orig.where(df > 0, -df, inplace=True)

In [307]:
df_orig

,A,B,C,D
2000-01-01,0.257600,1.599861,0.174971,0.404868
2000-01-02,0.157870,0.072178,1.362580,0.357375
2000-01-03,0.016534,0.166315,0.990686,1.797528
2000-01-04,0.079916,2.458044,1.164954,0.631615
2000-01-05,0.082609,2.486026,0.805138,0.743355
2000-01-06,2.567500,1.026044,0.653128,0.318010
2000-01-07,0.929976,0.391964,0.348340,0.188350
2000-01-08,0.194843,1.583846,0.458041,0.690173


__Note__: The signature for __DataFrame.where()__ differs from __numpy.where()__. Roughly `df1.where(m, df2)` is equivalent to `np.where(m, df1, df2)`.


In [308]:
df.where(df < 0, -df) == np.where(df <0, df, -df)

,A,B,C,D
2000-01-01,True,True,True,True
2000-01-02,True,True,True,True
2000-01-03,True,True,True,True
2000-01-04,True,True,True,True
2000-01-05,True,True,True,True
2000-01-06,True,True,True,True
2000-01-07,True,True,True,True
2000-01-08,True,True,True,True


#### alignment

Furthermore, `where` aligns the input boolean condition (ndarray or DataFrame), such that partial selection with setting is possible. This is analogous to partial setting via .loc (but on the contents rather than the axis labels).

In [312]:
dfm= df.copy()

In [313]:
dfm

,A,B,C,D
2000-01-01,0.257600,1.599861,0.174971,-0.404868
2000-01-02,-0.157870,0.072178,-1.362580,-0.357375
2000-01-03,-0.016534,0.166315,-0.990686,-1.797528
2000-01-04,-0.079916,-2.458044,1.164954,0.631615
2000-01-05,0.082609,-2.486026,-0.805138,-0.743355
2000-01-06,2.567500,1.026044,0.653128,-0.318010
2000-01-07,0.929976,0.391964,-0.348340,0.188350
2000-01-08,-0.194843,-1.583846,-0.458041,-0.690173


In [316]:
dfm[dfm[1:4] > 0]= 3 # Replace values where condition is True with 3

In [315]:
dfm

,A,B,C,D
2000-01-01,0.257600,1.599861,0.174971,-0.404868
2000-01-02,-0.157870,3.000000,-1.362580,-0.357375
2000-01-03,-0.016534,3.000000,-0.990686,-1.797528
2000-01-04,-0.079916,-2.458044,3.000000,3.000000
2000-01-05,0.082609,-2.486026,-0.805138,-0.743355
2000-01-06,2.567500,1.026044,0.653128,-0.318010
2000-01-07,0.929976,0.391964,-0.348340,0.188350
2000-01-08,-0.194843,-1.583846,-0.458041,-0.690173


Where can also accept `axis` and `level` parameters to align the input when performing the `where`.

In [274]:
dfm= df.copy()

In [276]:
dfm.where(dfm > 0, dfm['A'], axis='index') # Where value is > 0, replace with corresponding index value of col A

,A,B,C,D
2000-01-01,-0.117883,0.919023,-0.117883,-0.117883
2000-01-02,1.079590,0.496872,1.079590,0.405720
2000-01-03,-2.027375,0.497675,2.025348,0.330445
2000-01-04,0.056009,0.025912,0.056009,1.786789
2000-01-05,-0.410479,1.443175,1.354921,-0.410479
2000-01-06,1.686949,0.028588,1.686949,0.527568
2000-01-07,-1.728822,-1.728822,-1.728822,-1.728822
2000-01-08,-0.165180,-0.165180,-0.165180,-0.165180


This is equivalent to (but faster than) the following

In [277]:
dfm= df.copy()

In [279]:
# x= all rows, y= column to substitute values where filter is True
dfm.apply(lambda x, y: x.where(x > 0), y=dfm['A'])

,A,B,C,D
2000-01-01,NaN,0.919023,NaN,NaN
2000-01-02,1.079590,0.496872,NaN,0.405720
2000-01-03,NaN,0.497675,2.025348,0.330445
2000-01-04,0.056009,0.025912,NaN,1.786789
2000-01-05,NaN,1.443175,1.354921,NaN
2000-01-06,1.686949,0.028588,NaN,0.527568
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN


*New in version 0.18.1.*

Where can accept a `callable` as condition and other arguments. The function must be with one argument (the calling Series or DataFrame) and that returns valid output as condition and other argument.

In [282]:
df3= pd.DataFrame( {'A': range(1,4),
                   'B': range(4,7),
                   'C': range(7,10)})

In [283]:
df3

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [286]:
df3.where(lambda x: x < 4, lambda x: x+10) #For rows with values >= 4, add 10 to value

,A,B,C
0,1,14,17
1,2,15,18
2,3,16,19


# Mask

[mask()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mask.html#pandas.DataFrame.mask) is the inverse boolean operation of where.

`mask()`: replace values where condition is False

In [317]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [321]:
s.mask(s >=0) #For all values where condition is False, replace value with NaN?

4   NaN
3   NaN
2   NaN
1   NaN
0   NaN
dtype: float64

In [322]:
df

,A,B,C,D
2000-01-01,0.257600,1.599861,0.174971,-0.404868
2000-01-02,-0.157870,0.072178,-1.362580,-0.357375
2000-01-03,-0.016534,0.166315,-0.990686,-1.797528
2000-01-04,-0.079916,-2.458044,1.164954,0.631615
2000-01-05,0.082609,-2.486026,-0.805138,-0.743355
2000-01-06,2.567500,1.026044,0.653128,-0.318010
2000-01-07,0.929976,0.391964,-0.348340,0.188350
2000-01-08,-0.194843,-1.583846,-0.458041,-0.690173


In [323]:
df.mask(df >=0)

,A,B,C,D
2000-01-01,NaN,NaN,NaN,-0.404868
2000-01-02,-0.157870,NaN,-1.362580,-0.357375
2000-01-03,-0.016534,NaN,-0.990686,-1.797528
2000-01-04,-0.079916,-2.458044,NaN,NaN
2000-01-05,NaN,-2.486026,-0.805138,-0.743355
2000-01-06,NaN,NaN,NaN,-0.318010
2000-01-07,NaN,NaN,-0.348340,NaN
2000-01-08,-0.194843,-1.583846,-0.458041,-0.690173
